# Importing

In [1]:
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
from datetime import date, timedelta
from scipy import stats
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import warnings
warnings.filterwarnings('ignore')

# Data Importing

In [2]:
dtypes = {'store_nbr': np.dtype('int64'),
          'item_nbr': np.dtype('int64'),
          'unit_sales': np.dtype('float64'),
          'onpromotion': np.dtype('O')}

train = pd.read_csv('data/train.csv', dtype=dtypes)
test = pd.read_csv('data/test.csv', dtype=dtypes)
stores = pd.read_csv('data/stores.csv')
items = pd.read_csv('data/items.csv')
trans = pd.read_csv('data/transactions.csv')
oil = pd.read_csv('data/oil.csv')
holidays = pd.read_csv('data/holidays_events.csv')

MemoryError: Unable to allocate 957. MiB for an array with shape (1, 125497040) and data type float64

In [3]:
print("Shape of train:" , train.shape)
print("Shape of stores:" , stores.shape)
print("Shape of items:" , items.shape)
print("Shape of transactions:" , trans.shape)
print("Shape of oil:" , oil.shape)
print("Shape of holiday:" , holidays.shape)

NameError: name 'train' is not defined

In [4]:
# Checking Null Values
print("Undefined Values in Oil columns:",oil.isnull().any().values)
print("Undefined Values in holiday_events columns:",holidays.isnull().any().values)
print("Undefined Values in stores columns:",stores.columns.values,stores.isnull().any().values)
print("Undefined Values in transactions columns:",trans.isnull().any().values)

NameError: name 'oil' is not defined

In [ ]:
# drop any rows that contain missing values
oil.dropna(inplace=True)

In [ ]:
# Re-Checking Null Values
print("Undefined Values in Oil columns:",oil.isnull().any().values)

# Exploratory Data Analysis 

## Oil

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
ax.plot(oil['date'], oil['dcoilwtico'].dropna(), color='green', linewidth=2)
ax.fill_between(oil['date'], oil['dcoilwtico'].dropna(), 0, alpha=0.3, color='green')
ax.set_ylabel('Daily Oil price')
ax.set_title('Daily oil prices from Jan 2013 till July 2017')
plt.show()

Between January 2013 and July 2017, the chart illustrates that there has been a downward trend in the daily price of oil. Although the price of oil initially increased and even exceeded $100 in some months of 2013, it started to drop sharply in the middle of 2014, leading to a significant decrease in the price of oil. This trend is supported by some brief online research, which indicates that oil prices were relatively stable from 2010 until mid-2014, when they experienced a drastic decline. This was due to a combination of factors such as weak demand caused by slow economic growth and the rise of alternative sources of crude oil like shale/tar sands.

## Stores

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12))

sns.countplot(x="city", data=stores, ax=ax1)
sns.countplot(x="state", data=stores, ax=ax2)
sns.countplot(x="cluster", data=stores, ax=ax3)

plt.setp(ax1.xaxis.get_majorticklabels(), rotation=70)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=70)
plt.tight_layout()

Most of the stores are in Quito (from the Pichincha region) and Guayaquil (from the Guayas region) with Quito being the capital of Ecuador. As for the clusters they regroup stores of similar "type".

## Holiday Events

In [ ]:
f, axf = plt.subplots(figsize=(14, 8))
sns.countplot(x="locale_name", data=holidays, ax=axf)
plt.setp(axf.xaxis.get_majorticklabels(), rotation=80)
plt.xlabel('City')
plt.tight_layout()

## Transactions

In [ ]:
# Geographic visualization of sales by region
sales_by_region = pd.merge(trans, stores, on='store_nbr').groupby('state')['transactions'].sum()
sales_by_region.plot(kind='bar')
plt.title('Total Sales by Region')
plt.xlabel('Region')
plt.ylabel('Total Sales')
plt.show()

Pichincha has the highest sales

## Items

In [ ]:
f, ax = plt.subplots(figsize=(16, 8))
sns.countplot(x="family", data=items, ax=ax)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=80);

As we can see from the plot, the top 3 family categories are the GROCERY I, BEVERAGES and CLEANING categories.

# Data Preparation

In [ ]:
date_mask = (train['date'] >= '2017-07-15') & (train['date'] <= '2017-08-15')
pd_train = train[date_mask]

#Print the size
len(pd_train)

In [ ]:
#add missing date
min_oil_date = min(pd_train.date)
max_oil_date = max(pd_train.date)

calendar = []

d1 = datetime.datetime.strptime(min_oil_date, '%Y-%m-%d')  # start date date(2008, 8, 15)
d2 = datetime.datetime.strptime(max_oil_date, '%Y-%m-%d')  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    calendar.append(datetime.date.strftime(d1 + timedelta(days=i), '%Y-%m-%d'))

calendar = pd.DataFrame({'date':calendar})

oil = calendar.merge(oil, left_on='date', right_on='date', how='left')

In [ ]:
#Check how many NA
print(oil.isnull().sum(), '\n')

#Type
print('Type : ', '\n', oil.dtypes)

#Print the 3 first line
oil.head(5)

In [ ]:
na_index_oil = oil[oil['dcoilwtico'].isnull() == True].index.values
na_index_oil_plus = na_index_oil.copy()
na_index_oil_minus = np.maximum(0, na_index_oil-1)

for i in range(len(na_index_oil)):
    k = 1
    while (na_index_oil[min(i+k,len(na_index_oil)-1)] == na_index_oil[i]+k):
        k += 1
    na_index_oil_plus[i] = min(len(oil)-1, na_index_oil_plus[i] + k )

#Apply the formula
for i in range(len(na_index_oil)):
    if (na_index_oil[i] == 0):
        oil.loc[na_index_oil[i], 'dcoilwtico'] = oil.loc[na_index_oil_plus[i], 'dcoilwtico']
    elif (na_index_oil[i] == len(oil)):
        oil.loc[na_index_oil[i], 'dcoilwtico'] = oil.loc[na_index_oil_minus[i], 'dcoilwtico']
    else:
        oil.loc[na_index_oil[i], 'dcoilwtico'] = (oil.loc[na_index_oil_plus[i], 'dcoilwtico'] + oil.loc[na_index_oil_minus[i], 'dcoilwtico'])/ 2 

In [ ]:
#Merge train
pd_train = pd_train.drop('id', axis = 1)
pd_train = pd_train.merge(stores, left_on='store_nbr', right_on='store_nbr', how='left')
pd_train = pd_train.merge(items, left_on='item_nbr', right_on='item_nbr', how='left')
pd_train = pd_train.merge(holidays, left_on='date', right_on='date', how='left')
pd_train = pd_train.merge(oil, left_on='date', right_on='date', how='left')
pd_train = pd_train.drop(['description', 'state', 'locale_name', 'class'], axis = 1)

In [ ]:
#Shape
print('Shape : ', pd_train.shape, '\n')

#Type
print('Type : ', '\n', pd_train.dtypes)

#Summary
pd_train.describe()

In [ ]:
# Get the N most purchased products
def N_most_labels(data, variable , N , all='TRUE'):
    labels_freq_pd = stats.itemfreq(data[variable])
    labels_freq_pd = labels_freq_pd[labels_freq_pd[:, 1].argsort()[::-1]] #[::-1] ==> to sort in descending order
    
    if all == 'FALSE':
        main_labels = labels_freq_pd[:,0][0:N]
    else: 
        main_labels = labels_freq_pd[:,0][:]
        
    labels_raw_np = data[variable].to_numpy() #transform in numpy
    labels_raw_np = labels_raw_np.reshape(labels_raw_np.shape[0],1)

    labels_filtered_index = np.where(labels_raw_np == main_labels)
    
    return labels_freq_pd, labels_filtered_index

label_freq, labels_filtered_index = N_most_labels(data = pd_train, variable = "item_nbr", N = 10, all='FALSE')
print("labels_filtered_index[0].shape = ", labels_filtered_index[0].shape)

pd_train_filtered = pd_train.loc[labels_filtered_index[0],:]
print("pd_train_filtered.shape = ", pd_train_filtered.shape)

In [ ]:
label_freq[0:10]

In [ ]:
#Fill in cells if there is no holyday by the value : "no_holyday"
na_index_pd_train = pd_train_filtered[pd_train_filtered['type_y'].isnull() == True].index.values
print("Size of na_index_pd_train : ", len(na_index_pd_train), '\n')

pd_train_filtered.loc[pd_train_filtered['type_y'].isnull(), 'type_y'] = "no_holyday"
pd_train_filtered.loc[pd_train_filtered['locale'].isnull(), 'locale'] = "no_holyday"
pd_train_filtered.loc[pd_train_filtered['transferred'].isnull(), 'transferred'] = "no_holyday"
    
#check is there is NA
pd_train_filtered.isnull().sum()

In [ ]:
def get_month_year(df):
    df['month'] = df.date.apply(lambda x: x.split('-')[1])
    df['year'] = df.date.apply(lambda x: x.split('-')[0])
    
    return df

get_month_year(pd_train_filtered);

In [ ]:
pd_train_filtered['date'] = pd.to_datetime(pd_train_filtered['date'])
pd_train_filtered['day'] = pd_train_filtered['date'].dt.strftime
pd_train_filtered = pd_train_filtered.drop('date', axis=1)

In [ ]:
pd_train_filtered.sample(10)

In [ ]:
dummy_variables = ['onpromotion','city','type_x','cluster','store_nbr','item_nbr',
                'family','perishable','type_y', 'locale', 'transferred', 'month', 'day']

for var in dummy_variables:
    dummy = pd.get_dummies(pd_train_filtered[var], prefix = var, drop_first = False)
    pd_train_filtered = pd.concat([pd_train_filtered, dummy], axis = 1)

pd_train_filtered = pd_train_filtered.drop(dummy_variables, axis = 1)
pd_train_filtered = pd_train_filtered.drop(['year'], axis = 1)

In [ ]:
print('Shape : ', pd_train_filtered.shape)
pd_train_filtered.sample(10)

In [ ]:
#Re-scale
min_train, max_train = pd_train_filtered['unit_sales'].min(), pd_train_filtered['unit_sales'].max()
scalable_variables = ['unit_sales','dcoilwtico']

for var in scalable_variables:
    mini, maxi = pd_train_filtered[var].min(), pd_train_filtered[var].max()
    pd_train_filtered.loc[:,var] = (pd_train_filtered[var] - mini) / (maxi - mini)
print('Shape : ', pd_train_filtered.shape)
pd_train_filtered.sample(10)

In [ ]:
#train database without uni_sales
pd_train_filtered = pd_train_filtered.reset_index(drop=True)  #we reset the index
y_labels = pd_train_filtered['unit_sales']
X_train_filtered = pd_train_filtered.drop(['unit_sales'], axis = 1)

print('Shape X :', X_train_filtered.shape)
print('Shape y :', y_labels.shape)

In [ ]:
num_test = 0.20
X_train, X_validation, y_train, y_validation = train_test_split(X_train_filtered, y_labels, test_size=num_test, random_state=15)
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('X_validation shape :', X_validation.shape)
print('y_validation shape :', y_validation.shape)

# Model Creation

## Random Forest

In [ ]:
RFR = RandomForestRegressor()
parameters = {'n_estimators': [5, 10, 100],
              'min_samples_leaf': [1,5]
             }
grid_obj = GridSearchCV(RFR, parameters,
                        cv=5,
                        n_jobs=-1,
                        verbose=1)
grid_obj = grid_obj.fit(X_train, y_train)
RFR = grid_obj.best_estimator_
RFR.fit(X_train, y_train)
predictions = RFR.predict(X_validation)
print('R2 score = ',r2_score(y_validation, predictions))
print('MSE score = ',mean_squared_error(y_validation, predictions))

## Linear regression

In [ ]:
LR = LinearRegression()
LR.fit(X_train, y_train)
predictions = LR.predict(X_validation)
print('R2 score = ',r2_score(y_validation, predictions))
print('MSE score = ',mean_squared_error(y_validation, predictions))

## Ridge regression

In [ ]:
RR = Ridge(alpha=1.0)
RR.fit(X_train, y_train)
predictions = RR.predict(X_validation)
print('R2 score = ',r2_score(y_validation, predictions))
print('MSE score = ',mean_squared_error(y_validation, predictions))

## Keras Regression - Neural Network

In [ ]:
features = np.array(X_train)
targets = np.array(y_train.to_numpy().reshape(y_train.shape[0],1))
features_validation= np.array(X_validation)
targets_validation = np.array(y_validation.to_numpy().reshape(y_validation.shape[0],1))

# Building the model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(1))

# Compiling the model
model.compile(loss = 'mse', optimizer='adam', metrics=['mse']) #mse: mean_square_error
model.summary()

In [ ]:
# Training the model
epochs_tot = 1000
epochs_step = 250
epochs_ratio = int(epochs_tot / epochs_step)
hist =np.array([])

for i in range(epochs_ratio):
    history = model.fit(features, targets, epochs=epochs_step, batch_size=100, verbose=0)
    
    # Evaluating the model on the training and testing set
    print("Step : " , i * epochs_step, "/", epochs_tot)
    score = model.evaluate(features, targets)
    print("Training MSE:", score[1])
    score = model.evaluate(features_validation, targets_validation)
    print("Validation MSE:", score[1], "\n")

In [ ]:
predictions = model.predict(features_validation, verbose=0)

print('R2 score = ',r2_score(y_validation, predictions))
print('MSE score = ',mean_squared_error(y_validation, predictions))